### Oracle Guardian AI

In [2]:
#pip install oracle-guardian-ai[all-optional]
#! pip install scipy

In [3]:
import os
from guardian_ai.privacy_estimation.dataset import DataSplit, ClassificationDataset


In [4]:
from guardian_ai.privacy_estimation.model import (
    RandomForestTargetModel,
    GradientBoostingTargetModel,
    LogisticRegressionTargetModel,
    SGDTargetModel,
    MLPTargetModel
)
from guardian_ai.privacy_estimation.attack import AttackType
from guardian_ai.privacy_estimation.attack_runner import AttackRunner
from guardian_ai.privacy_estimation.plot_results import ResultPlot

In [5]:
# Source data directory
source_dir = ""
# dataset name
dataset_name = "titanic"
# source file
source_file = "titanic.csv"
# does the dataset contain header
contains_header = True
# index of the target variable
target_ix = 1
# Seed for data splits
data_split_seed = 42
# File to save results in
result_file = "titanic_out.txt"
# directory to store graphs
graph_dir = "."

In [6]:
if target_ix == -1:
    target_ix = None  # this will automatically pick the last index
    
ignore_ix = None  # specify if you need to ignore any features


# Define attack metrics we care about
metric_functions = ["precision", "recall", "f1", "accuracy"]
print_roc_curve = False  # print the values of the ROC curve

In [7]:
# Prepare result file for storing target model and attack metrics
fout = open(result_file, "w")
fout.write("dataset\tnum_rows\ttarget_model\ttrain_f1\ttest_f1\tattack_type")
for metric in metric_functions:
    fout.write("\tattack_" + metric)
fout.write("\n")


1

In [8]:
# Load data
print("Running Dataset: " + dataset_name)
dataset = ClassificationDataset(dataset_name)
dataset.load_data(os.path.join(source_dir,source_file),
                contains_header=contains_header,
                target_ix=target_ix,
                ignore_ix=ignore_ix)

Running Dataset: titanic


In [9]:
dataset

In [10]:
import pandas as pd

In [11]:
dsTitanic = pd.read_csv('titanic.csv')
dsTitanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
# string for reporting in the result file
result_dataset = dataset_name + "\t" + str(dataset.get_num_rows())

In [13]:
result_dataset

'titanic\t891'

In [14]:
dataset_split_ratios = {
    DataSplit.ATTACK_TRAIN_IN : 0.1,  # fraction of datapoints for training the
    # attack model, included in target model training set
    DataSplit.ATTACK_TRAIN_OUT : 0.1,  # fraction of datapoints for training the
    # attack model, not included in target model training set
    DataSplit.ATTACK_TEST_IN : 0.2,  # fraction of datapoints for evaluating the
    # attack model, included in target model training set
    DataSplit.ATTACK_TEST_OUT : 0.2,  # fraction of datapoints for evaluating the
    # attack model, not included in target model training set
    DataSplit.TARGET_ADDITIONAL_TRAIN : 0.1,  # fraction of datapoints included in
    # target model training set, not used in the attack training or testing
    DataSplit.TARGET_VALID : 0.1,  # fraction of datapoints for tuning the target model
    DataSplit.TARGET_TEST : 0.2  # fraction of datapoints for evaluating the
    # target model
}

In [15]:
dataset.prepare_target_and_attack_data(data_split_seed, dataset_split_ratios)

ATTACK_TRAIN_IN	89
ATTACK_TRAIN_OUT	89
ATTACK_TEST_IN	178
ATTACK_TEST_OUT	178
TARGET_ADDITIONAL_TRAIN	89
TARGET_VALID	89
TARGET_TEST	179


In [16]:
# Register target model
target_models = []
target_models.append(RandomForestTargetModel())
target_models.append(RandomForestTargetModel(n_estimators=1000))
target_models.append(GradientBoostingTargetModel())
target_models.append(GradientBoostingTargetModel(n_estimators=1000))
target_models.append(LogisticRegressionTargetModel())
target_models.append(SGDTargetModel())
target_models.append(MLPTargetModel())
target_models.append(MLPTargetModel(hidden_layer_sizes=(800,)))

In [17]:

# Specify which attacks you would like to run.
attacks = []
attacks.append(AttackType.LossBasedBlackBoxAttack)
#attacks.append(AttackType.ExpectedLossBasedBlackBoxAttack)
#attacks.append(AttackType.ConfidenceBasedBlackBoxAttack)
attacks.append(AttackType.ExpectedConfidenceBasedBlackBoxAttack)
attacks.append(AttackType.MerlinAttack)
#attacks.append(AttackType.CombinedBlackBoxAttack)
#attacks.append(AttackType.CombinedWithMerlinBlackBoxAttack)
attacks.append(AttackType.MorganAttack)

In [18]:
# Setup threshold grids for the threshold based attacks we plan to run.
threshold_grids = {
    AttackType.LossBasedBlackBoxAttack.name: [-0.0001, -0.001, -0.01, -0.05, -0.1, -0.3,
                                            -0.5, -0.7,-0.9, -1.0, -1.5, -10, -50, -100],
    AttackType.ConfidenceBasedBlackBoxAttack.name: [0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 0.9,
                                            0.99, 0.999, 1.0],
    AttackType.MerlinAttack.name: [0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99, 0.999, 1.0]
}

In [19]:
# Initiate AttackRunner
attack_runner = AttackRunner(dataset,
                        target_models,
                        attacks,
                        threshold_grids
                        )

attack_runner.train_target_models()

Target Model: random_forest_n_estimators_100
Target Model Train Evaluation: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       227
           2       1.00      1.00      1.00       129

    accuracy                           1.00       356
   macro avg       1.00      1.00      1.00       356
weighted avg       1.00      1.00      1.00       356

Target Model Test Evaluation: 
              precision    recall  f1-score   support

           1       0.84      0.91      0.87       121
           2       0.77      0.64      0.70        58

    accuracy                           0.82       179
   macro avg       0.81      0.77      0.79       179
weighted avg       0.82      0.82      0.82       179

Target Model: random_forest_n_estimators_1000
Target Model Train Evaluation: 
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       227
           2       1.00      1.00      1.00       1

In [20]:
# Set Cache
cache_input = AttackType.MorganAttack in attacks \
            or AttackType.CombinedBlackBoxAttack \
            or AttackType.CombinedWithMerlinBlackBoxAttack in attacks

In [21]:
for target_model in target_models:
    result_target = attack_runner.target_model_result_strings.get(target_model.get_model_name())
    print(result_target)
    print("target_model:",target_model)
    print("metric_functions:",metric_functions)
    print("print_roc_curve:",print_roc_curve)
    print("cache_input:",cache_input)

    for attack_type in attacks:
        print(attack_type)
        result_attack = attack_runner.run_attack(target_model,
                                                 attack_type,
                                                metric_functions,
                                                print_roc_curve=print_roc_curve,
                                                cache_input=cache_input)
        fout.write(result_dataset + "\t" + result_target + "\t" + result_attack)
    fout.flush()
fout.close()

random_forest_n_estimators_100	1.0	0.7855645402815215
target_model: <guardian_ai.privacy_estimation.model.RandomForestTargetModel object at 0x7f3973225460>
metric_functions: ['precision', 'recall', 'f1', 'accuracy']
print_roc_curve: False
cache_input: True
AttackType.LossBasedBlackBoxAttack
All grid-search results:
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': -0.0001}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': -0.001}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': -0.01}
precision: 0.622 (±0.088), recall: 0.338 (±0.055), f1: 0.434 (±0.056), for {'threshold': -0.05}
precision: 0.599 (±0.055), recall: 0.507 (±0.102), f1: 0.545 (±0.073), for {'threshold': -0.1}
precision: 0.569 (±0.051), recall: 0.742 (±0.142), f1: 0.639 (±0.075), for {'threshold': -0.3}
precision: 0.520 (±0.042), recall: 0.787 (±0.113), f1: 0.623 (±0.051), for {'threshold': -0.5}
precision: 

/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/guardian_ai/privacy_estimation/utils.py:78: RuntimeWarning: divide by zero encountered in log
  sample_loss = np.sum([-int(j==spos_dict[sample]) * np.log(y_pred[i][j]) for j in range(len(labels))])
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/guardian_ai/privacy_estimation/utils.py:78: RuntimeWarning: invalid value encountered in double_scalars
  sample_loss = np.sum([-int(j==spos_dict[sample]) * np.log(y_pred[i][j]) for j in range(len(labels))])
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWar

All grid-search results:
precision: 0.469 (±0.016), recall: 0.787 (±0.113), f1: 0.585 (±0.042), for {'threshold': 0.001}
precision: 0.469 (±0.016), recall: 0.787 (±0.113), f1: 0.585 (±0.042), for {'threshold': 0.01}
precision: 0.494 (±0.030), recall: 0.775 (±0.111), f1: 0.602 (±0.055), for {'threshold': 0.1}
precision: 0.530 (±0.040), recall: 0.697 (±0.075), f1: 0.601 (±0.050), for {'threshold': 0.3}
precision: 0.612 (±0.056), recall: 0.574 (±0.062), f1: 0.589 (±0.042), for {'threshold': 0.5}
precision: 0.562 (±0.112), recall: 0.380 (±0.091), f1: 0.452 (±0.099), for {'threshold': 0.7}
precision: 0.637 (±0.134), recall: 0.180 (±0.054), f1: 0.279 (±0.077), for {'threshold': 0.9}
precision: 0.200 (±0.400), recall: 0.011 (±0.022), f1: 0.021 (±0.042), for {'threshold': 0.99}
precision: 0.200 (±0.400), recall: 0.011 (±0.022), f1: 0.021 (±0.042), for {'threshold': 0.999}
precision: 0.200 (±0.400), recall: 0.011 (±0.022), f1: 0.021 (±0.042), for {'threshold': 1.0}
Models with a precision highe

/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/guardian_ai/privacy_estimation/utils.py:78: RuntimeWarning: divide by zero encountered in log
  sample_loss = np.sum([-int(j==spos_dict[sample]) * np.log(y_pred[i][j]) for j in range(len(labels))])
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/guardian_ai/privacy_estimation/utils.py:78: RuntimeWarning: invalid value encountered in double_scalars
  sample_loss = np.sum([-int(j==spos_dict[sample]) * np.log(y_pred[i][j]) for j in range(len(labels))])
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWar

All grid-search results:
precision: 0.468 (±0.019), recall: 0.787 (±0.113), f1: 0.585 (±0.047), for {'threshold': 0.001}
precision: 0.468 (±0.019), recall: 0.787 (±0.113), f1: 0.585 (±0.047), for {'threshold': 0.01}
precision: 0.494 (±0.032), recall: 0.775 (±0.111), f1: 0.602 (±0.056), for {'threshold': 0.1}
precision: 0.538 (±0.056), recall: 0.708 (±0.102), f1: 0.610 (±0.072), for {'threshold': 0.3}
precision: 0.592 (±0.043), recall: 0.584 (±0.056), f1: 0.587 (±0.044), for {'threshold': 0.5}
precision: 0.575 (±0.135), recall: 0.402 (±0.130), f1: 0.470 (±0.133), for {'threshold': 0.7}
precision: 0.737 (±0.253), recall: 0.167 (±0.092), f1: 0.262 (±0.133), for {'threshold': 0.9}
precision: 0.500 (±0.447), recall: 0.034 (±0.028), f1: 0.063 (±0.052), for {'threshold': 0.99}
precision: 0.500 (±0.447), recall: 0.034 (±0.028), f1: 0.063 (±0.052), for {'threshold': 0.999}
precision: 0.500 (±0.447), recall: 0.034 (±0.028), f1: 0.063 (±0.052), for {'threshold': 1.0}
Models with a precision highe

...

All grid-search results:
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.001}
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.01}
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.1}
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.3}
precision: 0.574 (±0.069), recall: 0.729 (±0.110), f1: 0.641 (±0.083), for {'threshold': 0.5}
precision: 0.400 (±0.490), recall: 0.022 (±0.027), f1: 0.042 (±0.052), for {'threshold': 0.7}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 0.9}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 0.99}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 0.999}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 1.0}
Models with a precision highe

/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       178
           1       0.50      1.00      0.67       178

    accuracy                           0.50       356
   macro avg       0.25      0.50      0.33       356
weighted avg       0.25      0.50      0.33       356

precision = 0.5
recall = 1.0
f1 = 0.6666666666666666
accuracy = 0.5
MerlinAttack	0.5	1.0	0.6666666666666666	0.5

AttackType.MorganAttack
All grid-search results:
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': -0.0001}
precision: 0.968 (±0.039), recall: 0.652 (±0.079), f1: 0.776 (±0.057), for {'threshold': -0.001}
precision: 0.929 (±0.051), recall: 1.000 (±0.000), f1: 0.962 (±0.027), for {'threshold': -0.01}
precision: 0.778 (±0.096), recall: 1.000 (±0.000), f1: 0.872 (±0.063), for {'threshold': -0.05}
precision: 0.663 (±0.054), recall: 1.000 (±0.000), f1: 0.796 (±0.040), for {'threshold': -0.1}
precision: 0.560 (±0.024), re

/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/datascience/conda/automlx_p38_cpu_v1/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/datascience/conda/automlx_p3

All grid-search results:
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.001}
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.01}
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.1}
precision: 0.500 (±0.009), recall: 1.000 (±0.000), f1: 0.667 (±0.008), for {'threshold': 0.3}
precision: 0.530 (±0.031), recall: 0.708 (±0.073), f1: 0.605 (±0.039), for {'threshold': 0.5}
precision: 0.600 (±0.490), recall: 0.034 (±0.028), f1: 0.064 (±0.053), for {'threshold': 0.7}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 0.9}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 0.99}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 0.999}
precision: 0.000 (±0.000), recall: 0.000 (±0.000), f1: 0.000 (±0.000), for {'threshold': 1.0}
Models with a precision highe

In [22]:
# Generates a plot
ResultPlot.print_best_attack(dataset_name=dataset.name,result_filename=result_file,graphs_dir=graph_dir,metric_to_sort_on="attack_accuracy")

Plotting dataset: titanic


<Figure size 640x480 with 0 Axes>